# Giới thiệu về Kỹ thuật Thiết kế Prompt
Kỹ thuật thiết kế prompt là quá trình xây dựng và tối ưu hóa các prompt cho các nhiệm vụ xử lý ngôn ngữ tự nhiên. Quá trình này bao gồm việc lựa chọn prompt phù hợp, điều chỉnh các tham số của chúng và đánh giá hiệu quả hoạt động. Kỹ thuật thiết kế prompt đóng vai trò quan trọng trong việc đạt được độ chính xác và hiệu suất cao cho các mô hình NLP. Trong phần này, chúng ta sẽ tìm hiểu những kiến thức cơ bản về kỹ thuật thiết kế prompt bằng cách sử dụng các mô hình của OpenAI để khám phá.


### Bài tập 1: Tách từ (Tokenization)
Khám phá cách tách từ bằng tiktoken, một công cụ tách từ mã nguồn mở, nhanh của OpenAI  
Xem thêm ví dụ tại [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Bài tập 2: Kiểm tra Thiết lập Khóa Github Models

Chạy đoạn mã dưới đây để xác minh rằng endpoint Github Models của bạn đã được thiết lập đúng cách. Đoạn mã này chỉ thử một prompt cơ bản và kiểm tra kết quả trả về. Đầu vào `oh say can you see` nên được hoàn thành theo kiểu như `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Bài tập 3: Thông tin bịa đặt

Khám phá điều gì xảy ra khi bạn yêu cầu LLM trả về các câu trả lời cho một chủ đề có thể không tồn tại, hoặc về những chủ đề mà nó có thể không biết vì nằm ngoài bộ dữ liệu đã được huấn luyện trước (gần đây hơn). Xem cách phản hồi thay đổi như thế nào nếu bạn thử một lời nhắc khác, hoặc một mô hình khác.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Bài tập 4: Dựa trên hướng dẫn  
Sử dụng biến "text" để đặt nội dung chính  
và biến "prompt" để đưa ra một hướng dẫn liên quan đến nội dung chính đó.

Ở đây, chúng ta yêu cầu mô hình tóm tắt đoạn văn cho một học sinh lớp hai


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Bài tập 5: Yêu cầu phức tạp
Hãy thử một yêu cầu có cả tin nhắn của hệ thống, người dùng và trợ lý
Hệ thống thiết lập ngữ cảnh cho trợ lý
Tin nhắn của người dùng & trợ lý tạo nên bối cảnh hội thoại nhiều lượt

Lưu ý cách tính cách của trợ lý được thiết lập là "mỉa mai" trong ngữ cảnh hệ thống.
Hãy thử sử dụng một ngữ cảnh tính cách khác. Hoặc thử một chuỗi tin nhắn đầu vào/đầu ra khác.


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Bài tập: Khám phá trực giác của bạn
Các ví dụ trên cung cấp cho bạn những mẫu mà bạn có thể sử dụng để tạo ra các prompt mới (đơn giản, phức tạp, hướng dẫn, v.v.) - hãy thử tạo thêm các bài tập khác để khám phá một số ý tưởng khác mà chúng ta đã thảo luận như ví dụ, gợi ý và nhiều hơn nữa.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
